In [1]:
import requests
from requests.exceptions import ConnectionError
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
companies1 = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1") #unmodified names
companies = pd.read_csv('new_quick_list.csv', encoding = "ISO-8859-1") #will have modified names

print(companies.dtypes)

companies['Security'] = companies['Security'].str.replace(u'\xa0', u' ')#this was messing with the classes
companies['Security'] = companies['Security'].str.replace(r' \(Class A\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class B\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class C\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r'\(', '')#need to remove parenthesis too for the url
companies['Security'] = companies['Security'].str.replace(r'\)', '')
companies['Security'] = companies['Security'].str.replace(r'\.com', '-com')#for amazon and salesforce
companies['Security'] = companies['Security'].str.replace(r'Corporation', 'corp')#corporation got shortened to corp
companies['Security'] = companies['Security'].str.replace(r'corporation', 'corp')
companies['Security'] = companies['Security'].str.replace(r'Company', 'co')#company got shortened to co
companies['Security'] = companies['Security'].str.replace(r'company', 'co')
companies['Security'] = companies['Security'].str.replace(r'&', 'and')#& is and for some reason
companies['Security'] = companies['Security'].str.replace(r' ', '-')#the url uses hyphens a lot
companies['Security'] = companies['Security'].str.replace(r'.', '')#don't want these here
companies['Security'] = companies['Security'].str.replace(r',', '')
companies['Security'] = companies['Security'].str.replace(r'!', '')
companies['Security'] = companies['Security'].str.replace(r'\'', '')

companies['Security'] = companies['Security'].str.lower()#lowercases everything
print(len(companies))
companies

Symbol                   object
Security                 object
SEC filings              object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date first added         object
CIK                       int64
Founded                  object
dtype: object
500


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3m-co,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,abbott-laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,abbvie-inc,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,abiomed-inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,accenture-plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
5,ATVI,activision-blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",8/31/2015,718877,2008
6,ADBE,adobe-inc,reports,Information Technology,Application Software,"San Jose, California",5/5/1997,796343,1982
7,AMD,advanced-micro-devices-inc,reports,Information Technology,Semiconductors,"Santa Clara, California",3/20/2017,2488,1969
8,AAP,advance-auto-parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",7/9/2015,1158449,1932
9,AES,aes-corp-the,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",10/2/1998,874761,1981


In [3]:
names_list = pd.read_csv('quick_names.csv', encoding = "ISO-8859-1")
names_list

,Security
0,3m-co
1,abbvie-inc
2,abiomed-inc
3,activision-blizzard
4,adobe-inc
5,advanced-micro-devices-inc
6,advance-auto-parts
7,aes-corp-the
8,aflac-inc
9,agilent-technologies-inc


In [4]:
#getting url this is for testing purposes
year = [2020, 2019, 2018, 2017]

for n in range(3):
    for y in range(len(year)):
        currName = companies.at[n, 'Security']
        url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
        url = 'https://www.salary.com/tools/executive-compensation-calculator/' + currName + '-executive-salaries?year=' + str(year[y])
        print(url)


https://www.salary.com/tools/executive-compensation-calculator/3m-co-executive-salaries?year=2020
https://www.salary.com/tools/executive-compensation-calculator/3m-co-executive-salaries?year=2019
https://www.salary.com/tools/executive-compensation-calculator/3m-co-executive-salaries?year=2018
https://www.salary.com/tools/executive-compensation-calculator/3m-co-executive-salaries?year=2017
https://www.salary.com/tools/executive-compensation-calculator/abbott-laboratories-executive-salaries?year=2020
https://www.salary.com/tools/executive-compensation-calculator/abbott-laboratories-executive-salaries?year=2019
https://www.salary.com/tools/executive-compensation-calculator/abbott-laboratories-executive-salaries?year=2018
https://www.salary.com/tools/executive-compensation-calculator/abbott-laboratories-executive-salaries?year=2017
https://www.salary.com/tools/executive-compensation-calculator/abbvie-inc-executive-salaries?year=2020
https://www.salary.com/tools/executive-compensation-calcu

In [5]:
#utilities
def getUrl(n, mod = ''):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + mod + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getIncUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-inc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCoUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-co' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCorpUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-corp' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getPlcUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-plc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getContent(page):
    doc=lh.fromstring(page.content)
    tr_elements=doc.xpath('//tr')
    return tr_elements

In [6]:
def failureCase(col, good, alt_tr_elements):
    for j in range(0,len(alt_tr_elements)):
        #T is our j'th row
        T=alt_tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                T=alt_tr_elements[j+1]
                #i is the index of our column
                i=0

                #this is a temporary storage of the content of each line of the csv
                content = []

                #Iterate through each element of the row
                for t in T.iterchildren():
                    #this part if for cleaning the cell contents
                    data=t.text_content() #gets the content of cell
                    data = data.__str__() #turns content to string
                    data = data.replace(r'$', '') #takes out dollar sign for number conversion
                    data = data.replace(r',', '') #takes out comma for number conversion
                    data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                    data = data.replace(r'CEO Pay', '') #cleans up the content string
                    data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                    data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                    data = data.replace(r'Ratio', '') #cleans up the content string
                    data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                    data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                    content.append(data)

                    #Check if row is empty
                    if i>0:
                    #Convert any numerical value to integers
                        try:
                            data=int(data)
                        except:
                            pass
                    #Append the data to col
                    col[i + 1][1].append(data)
                    #Increment i for the next column
                    i+=1
                #col[0][1].append(currName1)
                return True

        return False

In [7]:
#comp_names=[('Security', [])]

In [8]:
#Makes a list of to make the graph later on
col=[('Company Name', []), ('CEO Name', []), ('CEO Pay', []), ('Median Employee Pay', []), ('CEO Pay Ratio', []), ('Year', [])]
#col=[('Company Name', []), ('CEO Name', []), ('CEO Pay', []), ('Median Employee Pay', []), ('CEO Pay Ratio', [])]

In [ ]:
#ad something that adds ceo pay separately

In [9]:
#goes through all the webpages
year = [2020, 2019, 2018, 2017]
for n in range(len(names_list)):
    for y in range(len(year)):
        currName = names_list.at[n, 'Security']
        currName1 =companies1.at[n, 'Security']
        col[0][1].append(currName1)
        #getting url
        url = 'https://www.salary.com/tools/executive-compensation-calculator/' + currName + '-executive-salaries?year=' + str(year[y])
        page=requests.get(url)
        #getting all the tables
        tr_elements=getContent(page)
        #indicates that we failed to find the magics on this particular page
        fail = True
        for j in range(0,len(tr_elements)):
            #T is our j'th row
            T=tr_elements[j]

            #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
            #and CEO Pay Ratio)
            if len(T)==4:
                #checks if the current row has the 4 maagic words
                table_Name = ['','','','']
                i=0
                for t in T:
                    name = t.text_content()
                    name = name.__str__()
                    table_Name[i]=name
                    i+=1

                #if it does, I'll take the data from the row right below it
                if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                    #print(currName1 + ' good')
                    #setting fail to false
                    fail = False
                    T=tr_elements[j+1]
                    #i is the index of our column
                    i=0

                    #this is a temporary storage of the content of each line of the csv
                    content = []

                    #Iterate through each element of the row
                    for t in T.iterchildren():
                        #this part if for cleaning the cell contents
                        data=t.text_content() #gets the content of cell
                        data = data.__str__() #turns content to string
                        data = data.replace(r'$', '') #takes out dollar sign for number conversion
                        data = data.replace(r',', '') #takes out comma for number conversion
                        data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                        data = data.replace(r'CEO Pay', '') #cleans up the content string
                        data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                        data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                        data = data.replace(r'Ratio', '') #cleans up the content string
                        data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                        data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                        content.append(data)

                        #Check if row is empty
                        if i>0:
                        #Convert any numerical value to integers
                            try:
                                data=int(data)
                            except:
                                pass
                        #Append the data to col
                        col[i + 1][1].append(data)
                        #Increment i for the next column
                        i+=1
                    col[5][1].append(year[y])
                    break

        if fail:
            col[0][1].pop()
            col[1][1].append('na')
            col[2][1].append(-1)
            col[3][1].append(-1)
            col[4][1].append(-1)
            print(n, url, year[y])
        print(n,y)


0 https://www.salary.com/tools/executive-compensation-calculator/3m-co-executive-salaries?year=2020 2020
0 0
0 1
0 2
0 3
1 https://www.salary.com/tools/executive-compensation-calculator/abbvie-inc-executive-salaries?year=2020 2020
1 0
1 1
1 https://www.salary.com/tools/executive-compensation-calculator/abbvie-inc-executive-salaries?year=2018 2018
1 2
1 3
2 0
2 1
2 2
2 https://www.salary.com/tools/executive-compensation-calculator/abiomed-inc-executive-salaries?year=2017 2017
2 3
3 https://www.salary.com/tools/executive-compensation-calculator/activision-blizzard-executive-salaries?year=2020 2020
3 0
3 https://www.salary.com/tools/executive-compensation-calculator/activision-blizzard-executive-salaries?year=2019 2019
3 1
3 https://www.salary.com/tools/executive-compensation-calculator/activision-blizzard-executive-salaries?year=2018 2018
3 2
3 https://www.salary.com/tools/executive-compensation-calculator/activision-blizzard-executive-salaries?year=2017 2017
3 3
4 https://www.salary.com

32 1
32 2
32 3
33 https://www.salary.com/tools/executive-compensation-calculator/amgen-inc-executive-salaries?year=2020 2020
33 0
33 https://www.salary.com/tools/executive-compensation-calculator/amgen-inc-executive-salaries?year=2019 2019
33 1
33 2
33 3
34 https://www.salary.com/tools/executive-compensation-calculator/amphenol-corp-executive-salaries?year=2020 2020
34 0
34 https://www.salary.com/tools/executive-compensation-calculator/amphenol-corp-executive-salaries?year=2019 2019
34 1
34 2
34 3
35 https://www.salary.com/tools/executive-compensation-calculator/analog-devices-executive-salaries?year=2020 2020
35 0
35 1
35 2
35 https://www.salary.com/tools/executive-compensation-calculator/analog-devices-executive-salaries?year=2017 2017
35 3
36 https://www.salary.com/tools/executive-compensation-calculator/ansys-executive-salaries?year=2020 2020
36 0
36 https://www.salary.com/tools/executive-compensation-calculator/ansys-executive-salaries?year=2019 2019
36 1
36 https://www.salary.com

66 https://www.salary.com/tools/executive-compensation-calculator/blackrock-executive-salaries?year=2019 2019
66 1
66 https://www.salary.com/tools/executive-compensation-calculator/blackrock-executive-salaries?year=2018 2018
66 2
66 https://www.salary.com/tools/executive-compensation-calculator/blackrock-executive-salaries?year=2017 2017
66 3
67 https://www.salary.com/tools/executive-compensation-calculator/booking-holdings-inc-executive-salaries?year=2020 2020
67 0
67 https://www.salary.com/tools/executive-compensation-calculator/booking-holdings-inc-executive-salaries?year=2019 2019
67 1
67 2
67 3
68 https://www.salary.com/tools/executive-compensation-calculator/borgwarner-executive-salaries?year=2020 2020
68 0
68 https://www.salary.com/tools/executive-compensation-calculator/borgwarner-executive-salaries?year=2019 2019
68 1
68 https://www.salary.com/tools/executive-compensation-calculator/borgwarner-executive-salaries?year=2018 2018
68 2
68 https://www.salary.com/tools/executive-com

88 https://www.salary.com/tools/executive-compensation-calculator/celanese-executive-salaries?year=2019 2019
88 1
88 https://www.salary.com/tools/executive-compensation-calculator/celanese-executive-salaries?year=2018 2018
88 2
88 https://www.salary.com/tools/executive-compensation-calculator/celanese-executive-salaries?year=2017 2017
88 3
89 https://www.salary.com/tools/executive-compensation-calculator/centene-corp-executive-salaries?year=2020 2020
89 0
89 1
89 2
89 3
90 https://www.salary.com/tools/executive-compensation-calculator/centerpoint-energy-executive-salaries?year=2020 2020
90 0
90 https://www.salary.com/tools/executive-compensation-calculator/centerpoint-energy-executive-salaries?year=2019 2019
90 1
90 https://www.salary.com/tools/executive-compensation-calculator/centerpoint-energy-executive-salaries?year=2018 2018
90 2
90 https://www.salary.com/tools/executive-compensation-calculator/centerpoint-energy-executive-salaries?year=2017 2017
90 3
91 https://www.salary.com/too

112 https://www.salary.com/tools/executive-compensation-calculator/comcast-corp-executive-salaries?year=2019 2019
112 1
112 2
112 3
113 https://www.salary.com/tools/executive-compensation-calculator/comerica-inc-executive-salaries?year=2020 2020
113 0
113 https://www.salary.com/tools/executive-compensation-calculator/comerica-inc-executive-salaries?year=2019 2019
113 1
113 https://www.salary.com/tools/executive-compensation-calculator/comerica-inc-executive-salaries?year=2018 2018
113 2
113 https://www.salary.com/tools/executive-compensation-calculator/comerica-inc-executive-salaries?year=2017 2017
113 3
114 https://www.salary.com/tools/executive-compensation-calculator/conagra-brands-executive-salaries?year=2020 2020
114 0
114 https://www.salary.com/tools/executive-compensation-calculator/conagra-brands-executive-salaries?year=2019 2019
114 1
114 https://www.salary.com/tools/executive-compensation-calculator/conagra-brands-executive-salaries?year=2018 2018
114 2
114 https://www.salary

...

362 https://www.salary.com/tools/executive-compensation-calculator/prudential-financial-executive-salaries?year=2019 2019
362 1
362 https://www.salary.com/tools/executive-compensation-calculator/prudential-financial-executive-salaries?year=2018 2018
362 2
362 https://www.salary.com/tools/executive-compensation-calculator/prudential-financial-executive-salaries?year=2017 2017
362 3
363 https://www.salary.com/tools/executive-compensation-calculator/public-service-entrp-grp-inc-executive-salaries?year=2020 2020
363 0
363 1
363 2
363 3
364 https://www.salary.com/tools/executive-compensation-calculator/public-storage-executive-salaries?year=2020 2020
364 0
364 1
364 2
364 3
365 https://www.salary.com/tools/executive-compensation-calculator/pultegroup-executive-salaries?year=2020 2020
365 0
365 https://www.salary.com/tools/executive-compensation-calculator/pultegroup-executive-salaries?year=2019 2019
365 1
365 https://www.salary.com/tools/executive-compensation-calculator/pultegroup-executiv

390 1
390 2
390 3
391 https://www.salary.com/tools/executive-compensation-calculator/servicenow-executive-salaries?year=2020 2020
391 0
391 https://www.salary.com/tools/executive-compensation-calculator/servicenow-executive-salaries?year=2019 2019
391 1
391 https://www.salary.com/tools/executive-compensation-calculator/servicenow-executive-salaries?year=2018 2018
391 2
391 https://www.salary.com/tools/executive-compensation-calculator/servicenow-executive-salaries?year=2017 2017
391 3
392 https://www.salary.com/tools/executive-compensation-calculator/sherwin-williams-executive-salaries?year=2020 2020
392 0
392 https://www.salary.com/tools/executive-compensation-calculator/sherwin-williams-executive-salaries?year=2019 2019
392 1
392 https://www.salary.com/tools/executive-compensation-calculator/sherwin-williams-executive-salaries?year=2018 2018
392 2
392 https://www.salary.com/tools/executive-compensation-calculator/sherwin-williams-executive-salaries?year=2017 2017
392 3
393 https://ww

417 3
418 https://www.salary.com/tools/executive-compensation-calculator/thermo-fisher-scientific-executive-salaries?year=2020 2020
418 0
418 https://www.salary.com/tools/executive-compensation-calculator/thermo-fisher-scientific-executive-salaries?year=2019 2019
418 1
418 https://www.salary.com/tools/executive-compensation-calculator/thermo-fisher-scientific-executive-salaries?year=2018 2018
418 2
418 https://www.salary.com/tools/executive-compensation-calculator/thermo-fisher-scientific-executive-salaries?year=2017 2017
418 3
419 https://www.salary.com/tools/executive-compensation-calculator/tiffany-and-co-executive-salaries?year=2020 2020
419 0
419 https://www.salary.com/tools/executive-compensation-calculator/tiffany-and-co-executive-salaries?year=2019 2019
419 1
419 2
419 3
420 0
420 1
420 2
420 https://www.salary.com/tools/executive-compensation-calculator/tjx-cos-inc-the-executive-salaries?year=2017 2017
420 3
421 https://www.salary.com/tools/executive-compensation-calculator/tr

446 https://www.salary.com/tools/executive-compensation-calculator/verizon-communications-executive-salaries?year=2019 2019
446 1
446 https://www.salary.com/tools/executive-compensation-calculator/verizon-communications-executive-salaries?year=2018 2018
446 2
446 https://www.salary.com/tools/executive-compensation-calculator/verizon-communications-executive-salaries?year=2017 2017
446 3
447 https://www.salary.com/tools/executive-compensation-calculator/vertex-pharmaceuticals-inc-executive-salaries?year=2020 2020
447 0
447 https://www.salary.com/tools/executive-compensation-calculator/vertex-pharmaceuticals-inc-executive-salaries?year=2019 2019
447 1
447 2
447 3
448 https://www.salary.com/tools/executive-compensation-calculator/cbs-corp-executive-salaries?year=2020 2020
448 0
448 1
448 https://www.salary.com/tools/executive-compensation-calculator/cbs-corp-executive-salaries?year=2018 2018
448 2
448 3
449 https://www.salary.com/tools/executive-compensation-calculator/visa-inc-executive-

474 https://www.salary.com/tools/executive-compensation-calculator/xilinx-executive-salaries?year=2020 2020
474 0
474 https://www.salary.com/tools/executive-compensation-calculator/xilinx-executive-salaries?year=2019 2019
474 1
474 https://www.salary.com/tools/executive-compensation-calculator/xilinx-executive-salaries?year=2018 2018
474 2
474 https://www.salary.com/tools/executive-compensation-calculator/xilinx-executive-salaries?year=2017 2017
474 3
475 https://www.salary.com/tools/executive-compensation-calculator/xylem-inc-executive-salaries?year=2020 2020
475 0
475 https://www.salary.com/tools/executive-compensation-calculator/xylem-inc-executive-salaries?year=2019 2019
475 1
475 2
475 3
476 https://www.salary.com/tools/executive-compensation-calculator/yum-brands-inc-executive-salaries?year=2020 2020
476 0
476 1
476 2
476 3
477 https://www.salary.com/tools/executive-compensation-calculator/zebra-technologies-cp-cl-a-executive-salaries?year=2020 2020
477 0
477 1
477 2
477 3
478 ht

In [10]:
col

[('Company Name',
  ['3M Company',
   '3M Company',
   '3M Company',
   '3M Company',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'Accenture plc',
   'Accenture plc',
   'Accenture plc',
   'Accenture plc',
   'Activision Blizzard',
   'Activision Blizzard',
   'Activision Blizzard',
   'Activision Blizzard',
   'Adobe Inc.',
   'Adobe Inc.',
   'Adobe Inc.',
   'Adobe Inc.',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'AES Corp',
   'AES Corp',
   'AES Corp',
   'AES Corp',
   'AFLAC Inc',
   'AFLAC Inc',
   'AFLAC Inc',
   'AFLAC Inc',
   'Agilent Technologies Inc',
   'Agilent Technologies In

In [11]:
print(len(col[0][1]))
#print(col[0][1].pop())
#col[0][1].append('VF Corporation')
print(len(col[0][1]))
print(len(col[1][1]))
print(len(col[2][1]))
print(len(col[3][1]))
print(len(col[4][1]))
print(len(col[5][1]))

1924
1924
1924
1924
1924
1924
1924


In [12]:
#comp_names

In [13]:
#len(comp_names[0][1])
#Dict={title:column for (title,column) in comp_names}
#real_loc=pd.DataFrame(Dict)
#real_loc
#real_loc.to_csv('quick_names.csv',index=False)

In [14]:
#making the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [15]:
#displaying the dataframe
df

,Company Name,CEO Name,CEO Pay,Median Employee Pay,CEO Pay Ratio,Year
0,3M Company,na,-1,-1,-1,2020
1,3M Company,Michael F. Roman,18321566,57494,319,2019
2,3M Company,Michael F. Roman,17320459,57313,302,2018
3,3M Company,Inge G. Thulin,20494285,63338,324,2017
4,Abbott Laboratories,na,-1,-1,-1,2020
5,Abbott Laboratories,Richard A. Gonzalez,21610598,155885,139,2019
6,Abbott Laboratories,na,-1,-1,-1,2018
7,Abbott Laboratories,Richard A. Gonzalez,22625243,157347,144,2017
8,AbbVie Inc.,Michael R. Minogue,16612766,112393,148,2020
9,AbbVie Inc.,Michael R. Minogue,19243230,115794,166,2019


In [16]:
int = 0
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] > -1):
        int += 1
print(int)
print(len(df)-int)

808
1116


int = 0
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] == -1):
        int += 1
        print(i)
        print(df.at[i, 'Company Name'])
int

In [17]:
int = 0
for i in range(len(df)):
    if ('ney' in df.at[i, 'Company Name']):
        int += 1
        print(i)
int

920
921
922
923
1904
1905
1906
1907


8

In [18]:
df.at[230, 'Company Name']

'AvalonBay Communities'

In [19]:
companies.at[488, 'Security']

'williams-cos-inc'

In [20]:
companies1.at[487, 'Security']

'Whirlpool Corp.'

In [21]:
#verifying correctness

int = 0

for i in range(len(companies)):
    if (df.at[i, 'Company Name'] != companies1.at[i, 'Security']):
        int += 1
        print(df.at[i, 'Company Name'] + " is not " + companies1.at[i, 'Security'])
        print(i)
int

3M Company is not Abbott Laboratories
1
3M Company is not AbbVie Inc.
2
3M Company is not ABIOMED Inc
3
Abbott Laboratories is not Accenture plc
4
Abbott Laboratories is not Activision Blizzard
5
Abbott Laboratories is not Adobe Inc.
6
Abbott Laboratories is not Advanced Micro Devices Inc
7
AbbVie Inc. is not Advance Auto Parts
8
AbbVie Inc. is not AES Corp
9
AbbVie Inc. is not AFLAC Inc
10
AbbVie Inc. is not Agilent Technologies Inc
11
ABIOMED Inc is not Air Products & Chemicals Inc
12
ABIOMED Inc is not Akamai Technologies Inc
13
ABIOMED Inc is not Alaska Air Group Inc
14
ABIOMED Inc is not Albemarle Corp
15
Accenture plc is not Alexandria Real Estate Equities
16
Accenture plc is not Alexion Pharmaceuticals
17
Accenture plc is not Align Technology
18
Accenture plc is not Allegion
19
Activision Blizzard is not Alliant Energy Corp
20
Activision Blizzard is not Allstate Corp
21
Activision Blizzard is not Alphabet Inc.
22
Activision Blizzard is not Altria Group Inc
23
Adobe Inc. is not A

Cisco Systems is not TechnipFMC
431
Citigroup Inc. is not Teledyne Technologies
432
Citigroup Inc. is not Teleflex
433
Citigroup Inc. is not Teradyne
434
Citigroup Inc. is not Texas Instruments
435
Citizens Financial Group is not Textron Inc.
436
Citizens Financial Group is not Thermo Fisher Scientific
437
Citizens Financial Group is not Tiffany & Co.
438
Citizens Financial Group is not TJX Companies Inc.
439
Citrix Systems is not Tractor Supply Company
440
Citrix Systems is not Trane Technologies plc
441
Citrix Systems is not TransDigm Group
442
Citrix Systems is not The Travelers Companies Inc.
443
The Clorox Company is not Truist Financial
444
The Clorox Company is not Twitter, Inc.
445
The Clorox Company is not Tyler Technologies
446
The Clorox Company is not Tyson Foods
447
CME Group Inc. is not UDR, Inc.
448
CME Group Inc. is not Ulta Beauty
449
CME Group Inc. is not U.S. Bancorp
450
CME Group Inc. is not Under Armour
451
CMS Energy is not Union Pacific Corp
452
CMS Energy is not

499

In [22]:
#adding industry
print(list(companies1.columns))
industries = list(companies1['GICS\xa0Sector'])
('GICS Sector', industries)

['Symbol', 'Security', 'SEC filings', 'GICS\xa0Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK', 'Founded']


('GICS Sector',
 ['Industrials',
  'Health Care',
  'Health Care',
  'Health Care',
  'Information Technology',
  'Communication Services',
  'Information Technology',
  'Information Technology',
  'Consumer Discretionary',
  'Utilities',
  'Financials',
  'Health Care',
  'Materials',
  'Information Technology',
  'Industrials',
  'Materials',
  'Real Estate',
  'Health Care',
  'Health Care',
  'Industrials',
  'Utilities',
  'Financials',
  'Communication Services',
  'Consumer Staples',
  'Consumer Discretionary',
  'Materials',
  'Utilities',
  'Industrials',
  'Utilities',
  'Financials',
  'Financials',
  'Real Estate',
  'Utilities',
  'Financials',
  'Health Care',
  'Industrials',
  'Health Care',
  'Information Technology',
  'Information Technology',
  'Information Technology',
  'Health Care',
  'Financials',
  'Industrials',
  'Energy',
  'Real Estate',
  'Information Technology',
  'Information Technology',
  'Consumer Discretionary',
  'Consumer Staples',
  'Information

In [23]:
subIndustries = list(companies1['GICS Sub-Industry'])
('GICS Sub-Industry', subIndustries)

('GICS Sub-Industry',
 ['Industrial Conglomerates',
  'Health Care Equipment',
  'Pharmaceuticals',
  'Health Care Equipment',
  'IT Consulting & Other Services',
  'Interactive Home Entertainment',
  'Application Software',
  'Semiconductors',
  'Automotive Retail',
  'Independent Power Producers & Energy Traders',
  'Life & Health Insurance',
  'Health Care Equipment',
  'Industrial Gases',
  'Internet Services & Infrastructure',
  'Airlines',
  'Specialty Chemicals',
  'Office REITs',
  'Biotechnology',
  'Health Care Supplies',
  'Building Products',
  'Electric Utilities',
  'Property & Casualty Insurance',
  'Interactive Media & Services',
  'Tobacco',
  'Internet & Direct Marketing Retail',
  'Paper Packaging',
  'Multi-Utilities',
  'Airlines',
  'Electric Utilities',
  'Consumer Finance',
  'Property & Casualty Insurance',
  'Specialized REITs',
  'Water Utilities',
  'Asset Management & Custody Banks',
  'Health Care Distributors',
  'Electrical Components & Equipment',
  'Bi

In [24]:
col.append(('GICS Sector', industries))
col

[('Company Name',
  ['3M Company',
   '3M Company',
   '3M Company',
   '3M Company',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'Accenture plc',
   'Accenture plc',
   'Accenture plc',
   'Accenture plc',
   'Activision Blizzard',
   'Activision Blizzard',
   'Activision Blizzard',
   'Activision Blizzard',
   'Adobe Inc.',
   'Adobe Inc.',
   'Adobe Inc.',
   'Adobe Inc.',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'AES Corp',
   'AES Corp',
   'AES Corp',
   'AES Corp',
   'AFLAC Inc',
   'AFLAC Inc',
   'AFLAC Inc',
   'AFLAC Inc',
   'Agilent Technologies Inc',
   'Agilent Technologies In

In [25]:
col.append(('GICS Sub-Industry', subIndustries))
col

[('Company Name',
  ['3M Company',
   '3M Company',
   '3M Company',
   '3M Company',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'Abbott Laboratories',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'AbbVie Inc.',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'ABIOMED Inc',
   'Accenture plc',
   'Accenture plc',
   'Accenture plc',
   'Accenture plc',
   'Activision Blizzard',
   'Activision Blizzard',
   'Activision Blizzard',
   'Activision Blizzard',
   'Adobe Inc.',
   'Adobe Inc.',
   'Adobe Inc.',
   'Adobe Inc.',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advanced Micro Devices Inc',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'Advance Auto Parts',
   'AES Corp',
   'AES Corp',
   'AES Corp',
   'AES Corp',
   'AFLAC Inc',
   'AFLAC Inc',
   'AFLAC Inc',
   'AFLAC Inc',
   'Agilent Technologies Inc',
   'Agilent Technologies In

In [26]:
Dict={title:column for (title,column) in col}
df_ratio=pd.DataFrame(Dict)

ValueError: arrays must all be same length

In [ ]:
df_ratio

In [ ]:
#verifying correctness

int = 0

for i in range(len(companies)):
    if (df_ratio.at[i, 'GICS Sector'] != companies1.at[i, 'GICS\xa0Sector'] or df_ratio.at[i, 'GICS Sub-Industry'] != companies1.at[i, 'GICS Sub-Industry']):
        int += 1
        print(df.at[i, 'Company Name'] + " is not " + companies1.at[i, 'Security'])
        print(i)
int

In [ ]:
#df.to_csv('official_ratios.csv',index=False)

In [ ]:
#df_ratio.to_csv('official_ratio_with_industries.csv',index=False)

In [ ]:
int = []
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] == -1):
        int.append(i)
print(len(int))
int